In [1]:
from utilities import *
from plot_utilities import *

In [ ]:
def compute_prior_cdf(r, eta, n_samples = 1000, tail_bound = 0.05, n_tail = 5, scale = 1):
    
    r = np.round(r, 3)
    eta = np.round(eta, 3)
    if os.path.isfile(f'CDFs/optimize_cdfs_{n_samples}.pickle'):
        with open(f'CDFs/optimize_cdfs_{n_samples}.pickle', 'rb') as handle:
            cdfs = pickle.load(handle)
        if (r, eta) in cdfs:
            return cdfs[(r, eta)]
    else:
        cdfs = dict()

    beta = (eta + 1.5)/r 
    var_prior = scale * scipy.special.gamma((eta + 1.5 + 1/2)/r)/scipy.special.gamma(beta)
    x_max = min(100, np.round(var_prior/tail_bound)) 
    xs = np.linspace(-x_max, x_max, n_samples-2*n_tail)
    xs = np.append(np.linspace(-(x_max+100), -(x_max+20), n_tail), xs)
    xs = np.append(xs, np.linspace(x_max + 20, x_max + 100, n_tail))
    prior_pdf = np.full(xs.shape, np.nan)

    # Loop over xs
    for j, x in enumerate(xs):

        # Define integrands
        def gauss_density(theta):
            return (1./(np.sqrt(2*np.pi)*theta)) * np.exp(-0.5*(x/theta)**2)

        def gen_gamma_density(theta):
            return (r/scipy.special.gamma(beta)) * (1/scale) * (theta/scale)**(r*beta - 1) * np.exp(-(theta/scale)**r)

        def integrand(theta):
            return gauss_density(theta) * gen_gamma_density(theta)

        # Integrate 
        prior_pdf[j] = integrate.quad(integrand, 0, np.inf)[0]

    prior_cdf = np.zeros_like(prior_pdf)
    for i in range(len(xs) - 1):
        prior_cdf[i] = np.trapz(prior_pdf[:i+1], xs[:i+1]) 
    prior_cdf = np.append(prior_cdf[:-1], 1)

    poly = interpolate.CubicSpline(x = xs, y = prior_cdf)
    
    cdfs[(r, eta)] = poly
    with open(f'CDFs/optimize_cdfs_{n_samples}.pickle', 'wb') as handle:
        pickle.dump(cdfs, handle)
    return poly


In [6]:
cdfs_name = 'cdfs_100000_0.1-2.9-0.1_0-4-0.2'

with open(f'CDFs/{cdfs_name}.pickle', 'rb') as handle:
    all_cdfs = pickle.load(handle)
with open(f'panoptic/obs_x_dict.pickle', 'rb') as handle:
    obs_x_dict = pickle.load(handle)
with open(f'panoptic/df_dict_{cdfs_name}.pickle', 'rb') as handle:
    df_dict = pickle.load(handle)


In [ ]:
fixed_x = obs_x_dict[3]
n = len(fixed_x)

def kstest_stat(params):
    r = params[0]
    eta = params[1]
    cdf = compute_prior_cdf(r, eta, 1000)
    cdfvals = cdf(fixed_x)
    dplus, dminus = np.max(np.arange(1.0, n + 1) / n - cdfvals), np.max(cdfvals - np.arange(0.0, n)/n)
    return max(dplus, dminus)
kstest_params()

In [3]:
from scipy import optimize

In [ ]:
optimize.minimize